In [1]:
import nltk
import pandas as pd
from textblob import TextBlob
import logging
from gensim import corpora, models, similarities
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import os
import codecs
from sklearn import feature_extraction
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.learning_curve import learning_curve
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score,  roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_graphviz
from sklearn import svm, grid_search
from sklearn import tree
from sklearn.preprocessing import FunctionTransformer
import sklearn
import mpld3
import lda
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import sys
# reload(sys)  
# sys.setdefaultencoding('Cp1252')

In [3]:
bl=pd.read_csv('data/bl.csv', na_values='DK', encoding='latin-1')
fy15q3= pd.read_csv('data/fy15q3.csv', na_values='DK', encoding='latin-1')
fy16= pd.read_csv('data/fy16.csv', na_values='DK', encoding='latin-1')

In [4]:
len(fy16)

3068

### Data Cleaning

In [5]:
fy16['geo']=fy16['geo'].fillna('NaM')
fy15q3['Geo']=fy15q3['Geo'].fillna('NaM')
bl['Region : Region (Global Area)']=bl['Region : Region (Global Area)'].fillna('NaM')

In [6]:
fy16.loc[fy16['translation.to.english.for..ltr.comment'].isnull(),
       'comment'] = fy16['ltr.comment']

fy15q3.loc[fy15q3['translation_to_1_q_ca_ltrfollowup_reporting_comment'].isnull(),
       'comment'] = fy15q3['LTR Comment']


In [7]:
fy16_1=fy16[[ 'email', 'geo','segment', 'role2', 'ltr','acct.mgt',
      'license.contract', 'best.practice', 'implement', 'product.qual', 'functionality', 'ease.of.use', 
      'tech.support','fiscalyear', 'quarter', 'ltr.comment','ltr_comment_theme1', 'ltr_comment_theme2',
      'ltr_comment_theme3']]

In [8]:
fy15q3['responsedate'][1061:1064]='NaN'
fy15q3['responsedate']=pd.to_datetime(pd.Series(fy15q3['responsedate']), format="%m/%d/%y %H:%M")


/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
fy15q3['responseyr'], fy15q3['rmth'] = fy15q3['responsedate'].dt.year, fy15q3['responsedate'].dt.month

In [10]:
fy15q3['fiscalyear'] = 'NaN'
fy15q3.loc[fy15q3['rmth'] > 4, 'fiscalyear'] = "FY15"
fy15q3['quarter']="Q3"

In [11]:
fy15q3_1=fy15q3[[ 'Contact Email', 'Geo','Segment', 'Respondent Role', 'LTR', 
                 'INS - Overall Account Management',
      'INS - Licensing and Contract Process', 'INS - Best Practices Advice', 'INS - Product Implementation Process',
                 'INS - Overall Quality of Products', 'INS - Overall Functionality of Products', 
                 'INS - Overall Ease of Use', 
      'INS - Ability to Get Technical Support','fiscalyear', 'quarter', 'comment','LTR_Comment_Theme1 ',
                 'LTR_Comment_Theme2',
      'LTR_Comment_Theme3']]

In [12]:
fy15q3_1.columns=['email', 'geo','segment', 'role2', 'ltr','acct.mgt',
      'license.contract', 'best.practice', 'implement', 'product.qual', 'functionality', 'ease.of.use', 
      'tech.support','fiscalyear', 'quarter', 'ltr.comment','ltr_comment_theme1', 'ltr_comment_theme2',
      'ltr_comment_theme3']

In [13]:
bl.loc[bl['What can CA Technologies do to improve your likelihood to recommend us? - Translated'].isnull(),
       'new'] = bl['What can CA Technologies do to improve your likelihood to recommend us?']

In [14]:
bl.loc[bl['What one thing can CA Technologies to improve your likelihod to recommend us? Translated'].isnull(),
       'new1'] = bl['What one thing can CA Technologies to improve your likelihod to recommend us?']

In [15]:
bl.loc[bl['What can CA Technologies do to improve your likelihood to recommend us? - Translated.1'].isnull(),
       'new2'] = bl['What can CA Technologies do to improve your likelihood to recommend us?.1']

In [16]:
bl['ltr.comment']=pd.concat([bl['new'].dropna(), bl['new1'].dropna(), bl['new2'].dropna() ]).reindex_like(bl)
bl['theme1']=pd.concat([bl['LTR_Promoters_Theme1'].dropna(), bl['LTR_Passives_Theme1'].dropna(), bl['LTR_Detractors_Theme1'].dropna() ]).reindex_like(bl)
bl['theme2']=pd.concat([bl['LTR_Promoters_Theme2'].dropna(), bl['LTR_Passives_Theme2'].dropna(), bl['LTR_Detractors_Theme2'].dropna() ]).reindex_like(bl)
bl['theme3']=pd.concat([bl['LTR_Promoters_Theme3'].dropna(), bl['LTR_Passives_Theme3 : LTR_Passives_Theme3'].dropna(), bl['LTR_Detractors_Theme3'].dropna() ]).reindex_like(bl)

In [17]:
# bl.columns.tolist()

In [18]:
bl_1=bl[[ 'ContactEmail : ContactEmail', 'Region : Region (Global Area)','AccountType : AccountType',
         'Role_New_WP : Role', 'How likely are you to recommend CA Technologies ', 
        u'Overall Account Management\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6\xe6',
      'Licensing and contract process', 'Best-practices advice', 'Product implementation process',
                 'Overall quality of products', 'Overall functionality of products', 
                 'Overall ease of use of CA Technologies products', 'Ability to get technical support from CA Technologies',
         'Fiscal Year', 'Quarter', 
         'ltr.comment','theme1', 'theme2', 'theme3']]

In [19]:
bl_1.columns=['email', 'geo','segment', 'role2', 'ltr','acct.mgt',
      'license.contract', 'best.practice', 'implement', 'product.qual', 'functionality', 'ease.of.use', 
      'tech.support','fiscalyear', 'quarter', 'ltr.comment','ltr_comment_theme1', 'ltr_comment_theme2',
      'ltr_comment_theme3']

In [20]:
fy15q3_1['role2'] = fy15q3_1['role2'].map({'End User': 'END_USER', 'Influencer': 'DM/INF', 'Decision Maker': 'DM/INF', 'Recommender': 'END_USER'   })

/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
bl_1['role2'] = bl_1['role2'].map({'END': 'END_USER', 'INF': 'DM/INF', 'DM': 'DM/INF', 'PRO': 'END_USER'   })

/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [22]:
fy16_1['fiscalyear'].value_counts()

FY16    3068
Name: fiscalyear, dtype: int64

In [23]:
frames= [bl_1, fy15q3_1, fy16_1]
df=pd.concat(frames)

In [24]:
df = df[pd.notnull(df['ltr.comment'])]

In [25]:
df.columns

Index([u'email', u'geo', u'segment', u'role2', u'ltr', u'acct.mgt',
       u'license.contract', u'best.practice', u'implement', u'product.qual',
       u'functionality', u'ease.of.use', u'tech.support', u'fiscalyear',
       u'quarter', u'ltr.comment', u'ltr_comment_theme1',
       u'ltr_comment_theme2', u'ltr_comment_theme3'],
      dtype='object')

In [26]:
df = df[(df.geo == 'NaM') ]

In [27]:
df=df.fillna(df.mean())

In [28]:
df.head()

,email,geo,segment,role2,ltr,acct.mgt,license.contract,best.practice,implement,product.qual,functionality,ease.of.use,tech.support,fiscalyear,quarter,ltr.comment,ltr_comment_theme1,ltr_comment_theme2,ltr_comment_theme3
1,alain.z.cote@dgag.ca,NaM,Named,END_USER,0,9,6.772884,4,2,0,6,0.000000,0.000000,FY14,Q2,"Improve the quality, stability and integration...",Product_Quality,Product_Scope,NaN
2,matt.merchant@ge.com,NaM,Platinum,DM/INF,9,10,8.000000,9,10,9,9,7.313038,7.602971,FY14,Q2,ITSM space is best for CA. Still work to be do...,Product_Scope,NaN,NaN
3,chris.schwind@firstdata.com,NaM,Platinum,END_USER,9,10,6.772884,9,7,8,8,8.000000,8.000000,FY14,Q2,"Service Assurance, Mainframe tools, Scheduling...",Product_Scope,NaN,NaN
4,dominic.nguyen@ge.com,NaM,Platinum,END_USER,3,8,6.772884,7,3,2,4,2.000000,3.000000,FY14,Q2,Needs to get product more stable and less bugs,Product_Quality,NaN,NaN
5,mz1223@att.com,NaM,Platinum,END_USER,7,8,6.772884,10,10,9,10,8.000000,2.000000,FY14,Q2,We primarily use Introscope under the APM suit...,Contracts_Negotiations,NaN,NaN


In [29]:
df['license.contract'].value_counts()

6.772884     1768
8.000000      263
7.000000      197
9.000000      173
6.000000      132
10.000000     107
5.000000      103
4.000000       48
3.000000       42
2.000000       41
1.000000       26
0.000000       26
Name: license.contract, dtype: int64

In [30]:
df['ltr.comment']=df['ltr.comment'].str.replace(r'\?', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'\r', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'\#NAME', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'n/a', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'na', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'-', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'.', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'___', '')
df['ltr.comment']=df['ltr.comment'].str.replace(r'_', '')

In [31]:
df['fiscalyear'].value_counts()

FY16    1236
FY15     987
FY14     701
Name: fiscalyear, dtype: int64

In [32]:
df.columns.tolist()

['email',
 'geo',
 'segment',
 'role2',
 'ltr',
 'acct.mgt',
 'license.contract',
 'best.practice',
 'implement',
 'product.qual',
 'functionality',
 'ease.of.use',
 'tech.support',
 'fiscalyear',
 'quarter',
 'ltr.comment',
 'ltr_comment_theme1',
 'ltr_comment_theme2',
 'ltr_comment_theme3']

## Clustering

In [33]:
df.columns=[ 'email',
 'geo',
 'segment',
 'role2',
 'ltr',
 'acctmgt',
 'licensecontract',
 'bestpractice',
 'implement',
 'productqual',
 'functionality',
 'easeofuse',
 'techsupport',
 'fiscalyear',
 'quarter',
 'ltrcomment',
 'ltr_comment_theme1',
 'ltr_comment_theme2',
 'ltr_comment_theme3']

In [34]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords += ['.', ',', '(', ')', "'", '"', 'ca', 'customer', 'need', 'products', 'would', 'like', 'de', 'sure',
             'nothing', 'cost', 'us', 'use', 'get', 'make', 'needs', 'need', 'customer','customers',
              'would', 'get', 'like', '30', 'also', 'many', 'day'
             '30', '1000', '10', '06', '100', '1000', '1000s', '11', '12', '13','14', '15', '18', '19', '1sr', '200',
             '2005', '2009', '2011', '2012', '20126', '2014', '2015', 'vs', 'you', 'we', 'know', 'even', 'see', 'zuk', 
              'fen' , 'gartner', 'rules', 'cas', 'year', 'nothing', 'look', 'sure', 'solutions', 'better', 'sure' ,
              'line', 'one','clear', 'products', 'product', 'tool']

In [35]:
num_clusters = 50  # sp.unique(labels).shape[0]

import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')

from sklearn.feature_extraction.text import TfidfVectorizer


class StemmedTfidfVectorizer(TfidfVectorizer):

    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedTfidfVectorizer(min_df=10, max_df=0.5, ngram_range=(1, 5),
                                    stop_words=stopwords, decode_error='ignore'
                                    )
vectorized = vectorizer.fit_transform(df.ltrcomment)


In [36]:
feature= vectorized.toarray()
feature_df = pd.DataFrame(feature, columns=vectorizer.get_feature_names())
feature_df[:1]

,abil,abl,access,access mag,account,account mag,account rep,account team,accur,acquir,...,will,willing,within,without,work,work wel,workload,world,year,yet
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
word_frequency = feature_df.sum().reset_index()
word_frequency.columns=['word', 'freq']

word_frequency=word_frequency.sort(columns='freq', ascending= False)
word_frequency[:10]


/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,word,freq
610,support,159.482401
553,servic,109.603011
292,improv,95.190834
465,price,66.266098
700,work,60.142100
484,provid,59.170976
309,integr,56.479961
627,technolog,55.044660
320,issu,54.115881
620,team,52.515629


In [ ]:
from sklearn.cluster import KMeans
model = KMeans (n_clusters=30, init= 'k-means++', max_iter=100, n_init=1, verbose=3)

In [ ]:
model = KMeans (n_clusters=11, init= 'k-means++', max_iter=100, n_init=1 ,verbose=3)
model.fit(vectorized) #add sparse matrix 

In [ ]:
inertia = []
for i in range(2, 13):
    km = KMeans(n_clusters = i)
    km.fit(vectorized)
    inertia.append(km.inertia_)
    print (km.inertia_)
plt.plot(inertia)

In [ ]:
from sklearn.cluster import KMeans
model = KMeans (n_clusters=9, init= 'k-means++', max_iter=100, n_init=1, verbose=3)
model.fit(vectorized) #add sparse matrix 

In [ ]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(9):
   print "Cluster %d:" % i,
   for ind in order_centroids[i, :10]:
       print ' %s' % terms[ind],
   print

In [ ]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from pattern.en import tag
import nltk

n_topics = 11
n_top_words = 7


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

# Create custom tokenizer function that tags part of speech, takes only nouns, stems, and lemmatizes words
def noun_tokenize(text):
    lemmatizer = nltk.WordNetLemmatizer()
    stemmer = nltk.stem.porter.PorterStemmer()
    tokens = []
    for item in [token for token in nltk.tokenize.word_tokenize(text) if token not in stopwords]:
        word = stemmer.stem_word(item.lower())
        word = lemmatizer.lemmatize(word)
        tokens.append(word)
    return tokens



# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

import nltk.stem

english_stemmer = nltk.stem.SnowballStemmer('english')
stopwords = nltk.corpus.stopwords.words('english')
stopwords += ['.', ',', '(', ')', "'", '"', 'ca', 'customer', 'need', 'products', 'would', 'like', 'de', 'sure',
             'nothing', 'cost', 'us', 'use', 'get', 'make', 'needs', 'need', 'customer','customers',
              'would', 'get', 'like', '30', 'also', 'many', 'day'
             '30', '1000', '10', '06', '100', '1000', '1000s', '11', '12', '13','14', '15', '18', '19', '1sr', '200',
             '2005', '2009', '2011', '2012', '20126', '2014', '2015', 'vs', 'you', 'we', 'know', 'even', 'see', 'zuk', 
              'fen' , 'gartner', 'rules', 'cas', 'year', 'nothing', 'look', 'sure', 'solutions', 'better', 'sure' ,
              'line', 'one','clear', 'products', 'product', 'tool', '.', ',', '(', ')', "'", '"', ':', ';', '?', '!', 
              '"', "'", '%', '~', '/', '•', '’', '-', '_', '|', '@', ':', '[', ']']

from sklearn.feature_extraction.text import TfidfVectorizer


class StemmedTfidfVectorizer(TfidfVectorizer):

    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedTfidfVectorizer(min_df=10, max_df=0.5, ngram_range=(1, 3),
                                    stop_words=stopwords, decode_error='ignore'
                                    )
vectorized = vectorizer.fit_transform(df.ltrcomment)

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=60,
                                learning_method='online', learning_offset=50.,
                                random_state=888)
t0 = time()
lda.fit(vectorized)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
topics=pd.DataFrame(lda.transform(vectorized))

## Classification- Logistic Regression

In [ ]:
# df_new1=df.join(topics)

In [ ]:
df_new1=df

In [ ]:
fy=pd.get_dummies(df_new1['fiscalyear'])
geo=pd.get_dummies(df_new1['geo'])
role=pd.get_dummies(df_new1['role2'])
quarter=pd.get_dummies(df_new1['quarter'])
segment=pd.get_dummies(df_new1['segment'])
df1=pd.concat([fy, geo, role, quarter, segment, df_new1], axis=1)
df1=pd.concat([fy, role, quarter, segment, df_new1], axis=1)

In [ ]:
df1.columns.tolist()

In [ ]:
df1['ltr_comment_theme1'].value_counts()

In [ ]:
rename_theme=({'Product Quality': 'Product_Quality', 'Product_Quality': 'Product_Quality', 
               'Account_Team_Relationship': 'Account_Team_Relationship',
               'Contracts': 'Contracts','Product Mentions': 'Product Mentions',
               'Contracts_Negotiations': 'Contracts', 'Product_Scope': 'Product Scope',
            'Account Mgt': 'Account_Team_Relationship','Account Management and Agent Support': 'Support',
           'Support': 'Support','Product Scope': 'Product Scope','Brand': 'Brand','Documentation':'Product_Usability',
           'Issue Resolution': "Support",'Partners': "Brand",'Unable to recommend': "Unable to recommend",
            'Account_Team':'Account_Team_Relationship', 'Implementations/Upgrades': 'Implementations/Upgrades',
               'Implementations_Upgrades': 'Implementations/Upgrades',
            'CA Services': 'Services','Product Usability': 'Product_Usability',
               'Product_Usability': 'Product_Usability',
               'Self_service/KnowledgeBase/Website Topic': 'Self_service/KnowledgeBase/Website'})

In [ ]:
df1['theme1']=df1['ltr_comment_theme1'].map(rename_theme)
df1['theme2']=df1['ltr_comment_theme2'].map(rename_theme)
df1['theme3']=df1['ltr_comment_theme3'].map(rename_theme)

In [ ]:
df1['ltr_comment_theme3'].value_counts()

In [ ]:
df1['theme3'].value_counts()

In [ ]:
names=['Support', 'Account_Team_Relationship', 'Contracts', 'Product Scope', 'Product_Quality', 'Services', 
      'Brand', 'Implementations/Upgrades', 'Product_Usability', 'Documentation', 'Partners',
       'Self_service/KnowledgeBase/Website', 'Unable to recommend']

In [ ]:
for i in names:
    for column in df1:
        df1[i]=df1[i] = np.where((df1.theme1 ==i) | (df1.theme2 == i) | (df1.theme3==i), 1, 0)

In [ ]:
for i in names:
    for column in df1:
         test=df1[i].value_counts()

In [ ]:
df1['Support'].value_counts()

In [ ]:
df1['Account_Team_Relationship'].value_counts()

In [ ]:
df1['Contracts'].value_counts()

In [ ]:
df1['Product Scope'].value_counts()

In [ ]:
df1['Services'].value_counts()

In [ ]:
df1['Brand'].value_counts()

In [ ]:
df1['Implementations/Upgrades'].value_counts()

In [ ]:
df1['Product_Usability'].value_counts()

In [ ]:
df1['Documentation'].value_counts()

In [ ]:
df1['Partners'].value_counts()

In [ ]:
df1['Unable to recommend'].value_counts()

In [ ]:
rename_theme=({'Product Quality': 'Product_Quality', 'Product_Quality': 'Product_Quality', 
               'Account_Team_Relationship': 'Account_Team_Relationship',
               'Contracts': 'Contracts','Product Mentions': 'Product_Quality',
               'Contracts_Negotiations': 'Contracts', 'Product_Scope': 'Product_Quality',
            'Account Mgt': 'Account_Team_Relationship','Account Management and Agent Support': 'Support',
           'Support': 'Support','Product Scope': 'Product_Quality','Brand': 'Brand','Documentation':'Product_Quality',
           'Issue Resolution': "Support",'Partners': "Brand",'Unable to recommend': "Unable to recommend",
            'Account_Team':'Account_Team_Relationship', 'Implementations/Upgrades': 'Implementations/Upgrades',
               'Implementations_Upgrades': 'Implementations/Upgrades',
            'CA Services': 'Services','Product Usability': 'Product_Quality',
               'Product_Usability': 'Product_Quality',
               'Self_service/KnowledgeBase/Website Topic': 'Support'})

In [ ]:
df1['theme1']=df1['ltr_comment_theme1'].map(rename_theme)
df1['theme2']=df1['ltr_comment_theme2'].map(rename_theme)
df1['theme3']=df1['ltr_comment_theme3'].map(rename_theme)

In [ ]:
df1['theme1'].value_counts()

In [ ]:
df1.columns.tolist()

In [ ]:
new_data = df1.drop(['email', 'geo', 'segment', 'role2','ltrcomment', 'fiscalyear', 'FY14', 'FY15', 'FY16', 'quarter'
                    , 'ltr_comment_theme1', 'ltr_comment_theme2', 'ltr_comment_theme3', 'theme1', 'theme2', 
                    'theme3', 'Unable to recommend', 'Partners', 'Documentation', 
                     'Self_service/KnowledgeBase/Website'], axis=1)

In [ ]:
new_data.columns.tolist()

In [ ]:
# from sklearn.cross_validation import train_test_split
# X=new_data.iloc[:,0:53]
# y=new_data.iloc[:,54]
# # create 80%-20% train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
# est=LogisticRegression()
# est.fit(X_train, y_train)
# pred = est.predict(X_test)
# accuracy_score(pred, y_test)

In [ ]:
# scores = [accuracy_score, precision_score, recall_score]
# models = [LogisticRegression(), SVC(probability = True), GaussianNB(), DecisionTreeClassifier(max_depth = 4), 
#           RandomForestClassifier()]
# def get_metrics(features, outcome, test=0.2):
    
#     train_X, test_X, train_y, test_y = train_test_split(features, outcome, test_size = test, random_state=444)

    
#     # create empty lists
#     LogisticReg = []
#     SVMC = []
#     GaussNB = []
#     DecisionTree = []
#     RandomForest = []
#     kNN9 = []
#     ExtraTrees = []
    
#     # list of lists
#     lists = [LogisticReg, SVMC, GaussNB, DecisionTree, RandomForest]
    
#     # populate lists with scores of each scoring method
#     for i, model in enumerate(models):
#         for score in scores:
#             est = model
#             est.fit(train_X, train_y)
#             pred = est.predict(test_X)
#             lists[i].append(score(test_y, pred))
        
#     # create a dataframe which aggregates the lists
#     scores_df = pd.DataFrame(data = [LogisticReg, SVMC, GaussNB, DecisionTree, RandomForest])
#     scores_df.index = ["LogisticReg", "SVMC", "GaussNB", "DecisionTree", "RandomForest"]
#     scores_df.columns = ["Accuracy", "Precision", "Recall"]
#     return scores_df

In [ ]:
# get_metrics(X,y, 0.2)

## Classification - Naives Bayes/ SVM

In [ ]:
df4=df1[[
 'email',
 'ltrcomment',
 'theme1',
 'theme2',
 'theme3']]

In [ ]:
df5=pd.melt(df4, id_vars=[ 'email', 'ltrcomment'], value_vars=['theme1', 'theme2', 'theme3'], var_name='theme', 
            value_name='themes')

In [ ]:
df5[1:11]

In [ ]:
df5['themes1'] = pd.Categorical.from_array(df5.themes).codes
df5=df5.dropna()

In [ ]:
df2=df1[[u'FY14',
 u'FY15',
 u'FY16',
 'DM/INF',
 'END_USER',
 u'Q1',
 u'Q2',
 u'Q3',
 u'Q4',
 u'Existing_Enterprise',
 u'Growth',
 u'Named',
 u'New_Enterprise',
 u'Platinum',
 'email',
 'geo',
 'segment',
 'role2',
 'ltr',
 'acctmgt',
 'licensecontract',
 'bestpractice',
 'implement',
 'productqual',
 'functionality',
 'easeofuse',
 'techsupport',
 'fiscalyear',
 'quarter',
 'theme1',
 'theme2',
 'theme3']]

In [ ]:
df2.columns.tolist()

In [ ]:
df1.columns.tolist()

In [ ]:
df3=pd.melt(df2, id_vars=[ 'email','FY16','END_USER', 'Platinum'
                         ], value_vars=['theme1', 'theme2', 'theme3'], var_name='theme', value_name='themes')

In [ ]:
df4=df3.dropna()

In [ ]:
df4['themes'].value_counts()

In [ ]:
# 'Platinum',
#  'ltr',
#  'acctmgt',
#  'licensecontract',
#  'bestpractice',
#  'implement',
#  'productqual',
#  'functionality',
#  'easeofuse',
#  'techsupport'

In [ ]:
df4.head()

In [ ]:
len(df4)

In [ ]:
df4['themes'].value_counts()

In [ ]:
df4 = df4[(df4.themes != 'Unable to recommend') & (df4.themes != 'Services') & 
           (df4.themes != 'Implementations/Upgrades') & (df4.themes != 'Product Mentions') 
          & (df4.themes != 'Product Usability') & (df4.themes != 'Brand') & (df4.themes != 'Contracts')   ]

In [ ]:
df4['themes'].value_counts()

In [ ]:
df4['themes1'] = pd.Categorical.from_array(df4.themes).codes

In [ ]:
from sklearn.cross_validation import train_test_split
X=df4.iloc[:,1:3]
y=df4.iloc[:,6]
# create 80%-20% train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
scores = [accuracy_score, precision_score, recall_score]
models = [SVC(probability = True)]
def get_metrics(features, outcome, test=0.2):
    
    train_X, test_X, train_y, test_y = train_test_split(features, outcome, test_size = test, random_state=444)
    
    # create empty lists
    SVMC = []
    RandomForest = []
   
    # list of lists
    lists = [SVMC]    
    # populate lists with scores of each scoring method
    for i, model in enumerate(models):
        for score in scores:
            est = model
            est.fit(train_X, train_y)
            pred = est.predict(test_X)
            lists[i].append(score(test_y, pred))
        
    # create a dataframe which aggregates the lists
    scores_df = pd.DataFrame(data = [SVMC])
    scores_df.index = ["SVMC"]
    scores_df.columns = ["Accuracy", "Precision", "Recall"]
    return scores_df

In [ ]:
scores = [accuracy_score, precision_score, recall_score]
models = [RandomForestClassifier()]
def get_metrics1(features, outcome, test=0.2):
    
    train_X, test_X, train_y, test_y = train_test_split(features, outcome, test_size = test, random_state=444)
    
    # create empty lists
    RandomForest = []
   
    # list of lists
    lists = [RandomForest]    
    # populate lists with scores of each scoring method
    for i, model in enumerate(models):
        for score in scores:
            est = model
            est.fit(train_X, train_y)
            pred = est.predict(test_X)
            lists[i].append(score(test_y, pred))
        
    # create a dataframe which aggregates the lists
    scores_df = pd.DataFrame(data = [ RandomForest])
    scores_df.index = ["RandomForest"]
    scores_df.columns = ["Accuracy", "Precision", "Recall"]
    return scores_df


Baseline:

In [ ]:
get_metrics(X,y, 0.2)

In [ ]:
get_metrics1(X,y, 0.2)

## Random Forest

In [ ]:
from sklearn import grid_search
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=444)

rf = RandomForestClassifier(n_estimators=200)
parameters = {
              'max_depth':[5, 7, 9]}

clf_grid = grid_search.GridSearchCV(rf, parameters, n_jobs=-1)
clf_grid.fit(X_train, y_train)
test=clf_grid.best_estimator_

In [ ]:
scores = [accuracy_score, precision_score, recall_score]
models = [RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)]
def get_metrics1(features, outcome, test=0.2):
    
    train_X, test_X, train_y, test_y = train_test_split(features, outcome, test_size = test, random_state=444)
    
    # create empty lists
    RandomForest = []
   
    # list of lists
    lists = [RandomForest]    
    # populate lists with scores of each scoring method
    for i, model in enumerate(models):
        for score in scores:
            est = model
            est.fit(train_X, train_y)
            pred = est.predict(test_X)
            lists[i].append(score(test_y, pred))
        
    # create a dataframe which aggregates the lists
    scores_df = pd.DataFrame(data = [ RandomForest])
    scores_df.index = ["RandomForest"]
    scores_df.columns = ["Accuracy", "Precision", "Recall"]
    return scores_df


In [ ]:
get_metrics1(X,y,.2)

In [ ]:
scores = [accuracy_score, precision_score, recall_score]
models = [RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)]
def get_metrics1(features, outcome, test=0.2):
    
    train_X, test_X, train_y, test_y = train_test_split(features, outcome, test_size = test, random_state=444)
    
    # create empty lists
    RandomForest = []
   
    # list of lists
    lists = [RandomForest]    
    # populate lists with scores of each scoring method
    for i, model in enumerate(models):
        for score in scores:
            est = model
            est.fit(train_X, train_y)
            pred = est.predict(test_X)
            lists[i].append(score(test_y, pred))
        
    # create a dataframe which aggregates the lists
    scores_df = pd.DataFrame(data = [ RandomForest])
    scores_df.index = ["RandomForest"]
    scores_df.columns = ["Accuracy", "Precision", "Recall"]
    return pred

In [ ]:
test=get_metrics1(X,y,.2)

In [ ]:
test

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(model,x_train,y_train,x_test,y_test,cmap=plt.cm.Blues):
    y_pred = model.fit(x_train, y_train).predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm_normalized)
    plt.figure(figsize=(6,6))
    plt.imshow(cm_normalized, interpolation='nearest', cmap=cmap)
    plt.title('Normalized Confusion Matrix: {}')
    plt.colorbar()
    plt.xticks([0,1],['Negative diagnosis','Positive Diagnosis'])
    plt.yticks([0,1],['Negative diagnosis','Positive Diagnosis'])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(est,X_train,y_train,X_test,y_test,cmap=plt.cm.Blues)

In [ ]:
df4['themes'].value_counts()

In [ ]:
len(df4)

In [ ]:


# Feature Importance
from sklearn.ensemble import ExtraTreesClassifier
# load the iris datasets
# fit an Extra Trees model to the data
model = ExtraTreesClassifier()
model.fit(X, y)
# display the relative importance of each attribute
print(model.feature_importances_)

In [ ]:
# Recursive Feature Elimination
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
# load the iris datasets
dataset = datasets.load_iris()
# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 3)
rfe = rfe.fit(X,y)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
from sklearn.grid_search import GridSearchCV
model = RandomForestClassifier()
parameters = [{"n_estimators": [250, 500, 1000]}]
 
# Returns the best configuration for a model using crosvalidation
# and grid search
def best_config(model, parameters, train_instances, judgements):
    clf = GridSearchCV(model, parameters, cv=5,
                       scoring="accuracy", verbose=5, n_jobs=4)
    clf.fit(train_instances, judgements)
    best_estimator = clf.best_estimator_
    print('Best hyperparameters: ' + str(clf.best_params_))
 
    return [str(clf.best_params_), clf.best_score_,
            best_estimator]


In [ ]:
best_config(model, parameters, X_train, y_train)

In [ ]:
from sklearn.cross_validation import train_test_split
X=df5.iloc[:,1]
y=df5.iloc[:,4]
# create 80%-20% train-test split
X_train_tdf, X_test_tdf, y_train_tdf, y_test_tdf = train_test_split(X, y, test_size=0.2)


In [ ]:
import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')

from sklearn.feature_extraction.text import CountVectorizer


class StemmedTfidfVectorizer(CountVectorizer):

    def build_analyzer(self):
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedTfidfVectorizer(min_df=0, max_df=0.5, ngram_range=(1, 5),
                                    stop_words=stopwords, decode_error='ignore'
                                    )
X_train_counts = vectorizer.fit_transform(X_train_tdf)



In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

## Naives Bayes Multinomial

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train_tdf)

In [ ]:
X_new_counts = vectorizer.transform(X_test)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),  ('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])

In [ ]:
text_clf = text_clf.fit(X_train_tdf, y_train_tdf)

In [ ]:
import numpy as np
docs_test = X_test
predicted = text_clf.predict(X_test_tdf)
np.mean(predicted == y_test_tdf)     

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))
#metrics.confusion_matrix(y_test, predicted)

## Ensemble

In [ ]:
from brew.base import Ensemble
from brew.base import EnsembleClassifier
from brew.combination import  Combiner

# create your Ensemble
clfs = your_list_of_classifiers # [clf1, clf2]
ens = Ensemble(classifiers = clfs)

# create your Combiner
# the rules can be 'majority_vote', 'max', 'min', 'mean' or 'median'
comb = Combiner(rule='majority_vote')

# now create your ensemble classifier
ensemble_clf = EnsembleClassifier(ensemble=ens, combiner=comb)
ensemble_clf.predict(X)